# 01_parse_insitu

This notebook harmonizes insitu temperature data from water stations to a common format. The output files in /data/parsed are expected to have the following columns:

- **dt64**:         datetime64 formatted timestamp
- **station_id**:   station identifier
- **station_name**: station name
- **temperature**:  insitu temperature value
- **river**:        associted river name
- **geometry**:     wgs84 point coordinate of sampling position

In [2]:
import pandas as pd
import geopandas as gpd
import hvplot.pandas
from pathlib import Path

## NRW_HYGON data

In [50]:
# Read station geo-metadata
path_stations = Path('data/insitu/nrw_hygon/temp_stationen.txt')
df_stations = pd.read_csv(path_stations, sep=';')
gdf_stations = gpd.GeoDataFrame(
    df_stations, geometry=gpd.points_from_xy(x=df_stations.station_longitude, y=df_stations.station_latitude)
)

# Read data
path_data = Path('data/insitu/nrw_hygon/temp_messwerte.txt')
df_data = pd.read_csv(path_data, sep=';')
df_data['dt64'] = pd.to_datetime(df_data['time'])

# Filter stations (Rhine)
gdf_stations_rhine = gdf_stations.loc[gdf_stations.river_name == 'Rhein']
df_data = df_data.loc[df_data.station_no.isin(gdf_stations_rhine.station_no)]

,station_latitude,station_longitude,station_name,station_no,catchment_no,catchment_name,river_name,station_diary,LANUV_Eindat,CATCHMENT_SIZE,...,ObjectDescription,LANUV_MN7W,LANUV_N7W,LANUV_AQZmstnr,LANUV_AQZname,INTERNET_BEMERKUNG,LANUV_weitererBetreiber,station_status_remark,station_status,geometry
0,50.895665,8.023180,Weidenau2,2721490000100,272,Siegeinzugsgebiet Östlich,Ferndorfbach,NaN,22.08.1990,"153,00 km²",...,NaN,"29,3","28,1",NaN,NaN,NaN,NaN,NaN,NaN,POINT (8.02318 50.89567)
1,51.911005,9.130011,Schieder-Nessenberg,4567000000100,4,Weserzuflüsse,Emmer,NaN,01.11.1980,"267,22 km²",...,NaN,"60,8","58,5",NaN,NaN,NaN,NaN,NaN,NaN,POINT (9.13001 51.91101)
2,52.016211,8.959631,Vossheide,4621900000100,4,Weserzuflüsse,Bega,NaN,01.11.1962,"82,20 km²",...,NaN,"38,6","36,5",NaN,NaN,NaN,WWV,NaN,NaN,POINT (8.95963 52.01621)
3,51.199449,6.229859,Pannenmuehle,2847500000100,286,Niers- und Schwalmeinzugsgebiet,Schwalm,NaN,01.11.1930,"130,19 km²",...,NaN,"28,4","24,7",NaN,NaN,NaN,NaN,NaN,NaN,POINT (6.22986 51.19945)
4,50.910725,7.193176,Hoffnungsthal,2728893000100,272,Siegeinzugsgebiet Westlich,Sülz,NaN,01.06.1949,"219,00 km²",...,NaN,"41,4","37,9",NaN,NaN,NaN,NaN,NaN,NaN,POINT (7.19318 50.91072)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,51.135760,6.291689,Baltes,2842000000100,286,Niers- und Schwalmeinzugsgebiet,Beeckbach,NaN,01.11.1985,"19,65 km²",...,NaN,"17,8","16,8",NaN,NaN,NaN,NaN,NaN,NaN,POINT (6.29169 51.13576)
74,51.169289,8.671682,Medelon_W,4284170000100,428,Edereinzugsgebiet,Orke,NaN,NaN,"33,98 km²",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (8.67168 51.16929)
75,51.453643,9.115452,Welda,4449900000100,44,Diemeleinzugsgebiet,Twiste,NaN,01.08.1968,"433,73 km²",...,NaN,"38,0","35,4",NaN,NaN,NaN,NaN,NaN,NaN,POINT (9.11545 51.45364)
76,50.776982,7.442692,Eitorf,2725910000100,272,Siegeinzugsgebiet Westlich,Sieg,NaN,01.11.1967,"1468,00 km²",...,NaN,"17,6","14,9",NaN,NaN,NaN,NaN,NaN,NaN,POINT (7.44269 50.77698)


## BW_LUBW data

In [112]:
# Read data
path_data = Path('data/insitu/bw_lubw/lubw_rhine_stations.csv')
df_data = pd.read_csv(path_data, sep=',', encoding='ANSI', skiprows=8)
df_data['dt64'] = pd.to_datetime(df_data['Datum'], format=r'%d.%m.%Y')
df_data = df_data.dropna()
df_data['Tagesmittelwert'] = df_data['Tagesmittelwert'].apply(lambda x: float(x.replace(',', '.')))
gdf_data = gpd.GeoDataFrame(
    df_data, geometry=gpd.points_from_xy(x=df_data.Ost, y=df_data.Nord, crs='epsg:25832')
).to_crs('epsg:4326')
gdf_data = gdf_data \
    .rename(columns={'GCODE': 'station_id', 'Gewässer':'river', 
                     'Tagesmittelwert': 'temperature', 'Messstelle':'name'}) \
    .drop(columns=['Ost', 'Nord', 'Parameter', 'Datum', 'Dimension'])

# Sampling geometries at center of river
df = pd.DataFrame(
    {
        "station_id": ['CXX147', 'CXX334', 'CXX359'],
        "Latitude": [47.57718044047036, 48.83126839719035, 49.01138576721586],
        "Longitude": [7.81360143097896, 8.109691899961788, 8.29631894710105],
    }
)
gdf_sampling = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:4326"
).drop(columns=['Longitude', 'Latitude'])

# Replace geometries with LST sampling
gdf_data = gdf_data.drop(columns=['geometry']).merge(gdf_sampling, on='station_id')

# Export
path_data_out = Path('data/insitu/parsed/lubw_rhine_stations.csv')
gdf_data.to_csv(path_data_out)

## BS_opendataswiss data

Kontinuierlich gemessene Parameter der Rheinüberwachungsstation Weil am Rhein (RUES), jeweils gemittelt über 15 Minuten.  Werte werden alle 15 Minuten wie von den Sensoren gemessen als Rohdaten publiziert. Jährlich werden die Daten manuell plausibilisiert und als Stundenwerte veröffentlicht. 

Standort: Rheinkilometer 171.37km
Koordinaten: E 7.594868 N 47.601299 bzw. E 7° 35‘ 39.55“ N 47° 36‘ 4.68“ bzw. 611740 / 272310

In [10]:
# Read data
path_data = Path('data/insitu/bs_opendataswiss/weilamrein.csv')
df_data = pd.read_csv(path_data, sep=';')
df_data['dt64'] = pd.to_datetime(df_data['Startzeitpunkt'])+pd.Timedelta(0.5, 'h')
df_data['lat'] = 47.60470917070526
df_data['lon'] = 7.590989847821154
df_data['station_id'] = 'bs_war'
df_data['station_name'] = 'Weil am Rhein'
df_data['river'] = 'Rhine'
df_data = df_data.dropna().sort_values('dt64')
gdf_data = gpd.GeoDataFrame(
    df_data, geometry=gpd.points_from_xy(x=df_data.lon, y=df_data.lat, crs='epsg:4326')
)
gdf_data = gdf_data \
    .rename(columns={'Temperatur [°C]': 'temperature'}) \
    .drop(columns=['Startzeitpunkt', 'Start_text', 'Endezeitpunkt', 'Ende_text', 'Elektrische Leitfähigkeit [µS/cm]', 'Sauerstoffgehalt [mg/L]', 'pH-Wert'])

# Export
path_data_out = Path('data/insitu/parsed/bs_rhine_station.csv')
gdf_data.to_csv(path_data_out)

C:\Users\Michael\AppData\Local\Temp\ipykernel_31640\186580377.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_data['dt64'] = pd.to_datetime(df_data['Startzeitpunkt'])+pd.Timedelta(0.5, 'h')
